In [1]:
import os
from minio import Minio

import pyspark
from pyspark import SparkFiles
from pyspark.sql import SparkSession, functions as F, types
from datetime import datetime, timedelta

In [2]:
client = Minio(
    'localhost:9000',
    access_key=os.getenv('DEV_USER_ACCESS_KEY'),
    secret_key=os.getenv('DEV_USER_SECRET_KEY'),
    secure=False
)

bucket_name = 'local-data-lake-bronze'

found = client.bucket_exists(bucket_name)
if not found:
    client.make_bucket(bucket_name)

In [3]:
spark = SparkSession.builder \
    .master('local[5]') \
    .appName('local-data-lake') \
    .config('spark.jars', '../jars/hadoop-aws-3.3.4.jar,../jars/aws-java-sdk-bundle-1.12.262.jar') \
    .getOrCreate()

sc = spark.sparkContext
print(spark.sparkContext._jsc.sc().listJars())
print(f'The Pyspark version {spark.version} is running...')

24/05/30 21:35:58 WARN Utils: Your hostname, Denniss-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.6 instead (on interface en0)
24/05/30 21:35:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/05/30 21:35:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Vector(spark://192.168.0.6:59902/jars/aws-java-sdk-bundle-1.12.262.jar, spark://192.168.0.6:59902/jars/hadoop-aws-3.3.4.jar)
The Pyspark version 3.3.4 is running...


In [4]:
sc._jsc.hadoopConfiguration().set('fs.s3a.access.key', os.getenv('DEV_USER_ACCESS_KEY'))
sc._jsc.hadoopConfiguration().set('fs.s3a.secret.key', os.getenv('DEV_USER_SECRET_KEY'))
sc._jsc.hadoopConfiguration().set('fs.s3a.endpoint', 'http://127.0.0.1:9000')
sc._jsc.hadoopConfiguration().set('fs.s3a.path.style.access', 'true')
sc._jsc.hadoopConfiguration().set('fs.s3a.connection.ssl.enabled', 'false')
sc._jsc.hadoopConfiguration().set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
sc._jsc.hadoopConfiguration().set('fs.s3a.connection.ssl.enabled', 'false')

In [5]:
def load_taxi_data(dt):
    url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{dt}.parquet'
    sc.addFile(url)
    
    df = spark.read.parquet(SparkFiles.get(f'yellow_tripdata_{dt}.parquet'))
    
    return df

In [6]:
dt = '2024-03'

df = load_taxi_data(dt)
df.count()

df \
    .write \
    .mode('overwrite') \
    .parquet(f's3a://local-data-lake-bronze/{dt}/yellow_taxi.parquet')

24/05/30 21:36:25 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [ ]:
spark.stop()